## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [3]:
# !pip install gensim

  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101346 sha256=d9657a8cd3a91eb63cab9efe5166f3e66d784f6f68634b786f38e44c900f19ab
  Stored in directory: c:\users\영현\appdata\local\pip\cache\wheels\16\64\85\f3205b74e01a98fb81e081c0d61c2ecd04e4645a986db3726e
Successfully built smart-open
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15


In [4]:
from tqdm import tqdm_notebook # progress bar
from konlpy.tag import Mecab # Mecab 형태소 분석기 불러오기
mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")

import string # 특수문자
import warnings # 경고 알림 제거를 위한 라이브러리

# gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다.
from gensim import corpora # LDA를 돌릴 수 있는 형태로 바꿔주는 모듈
from gensim import models # 여기에 LDA가 들어있음

import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시

### 2. 텍스트 전처리 함수 만들기

- 전에 네이버 기사 pk파일로 저장했었음
- 네이버 기사를 페이지 단위로 가져왔기때문에 페이지 리스트(2차원)로 되어있음
- 일단, 리스트 1개에 토픽모델링에 넣을 모든 문서가 다 들어가있어서 1차원 리스트로 바꿔줘야해

In [29]:
def read_documents(input_file_name):
    
    corpus = []

    # pk 파일을 읽어서 리스트로 변환하여 돌려줌.
    with open(input_file_name, 'rb') as f : # rb:read as binary (pickle파일 읽기위해서)
        temp_corpus = pickle.load(f) # 2차원 리스트, 페이지 당 리스트 1개(뉴스기사 1개 -> 리스트 1개)
    
    for page in temp_corpus:
        corpus += page # 하나로 붙여줄거야, 긴 1차원 리스트

    return corpus

def text_cleaning(docs):
    # 전에 짰던 text_cleaning함수는 string하나를 받았었어
    # 지금은 리스트기때문에 변형시켜야 해
    # 한국어를 제외한 글자를 제거하는 함수를 편의를 위해 조금 변형해보자.
    cleaned_docs = []
    
    for doc in docs: # 리스트 1개에 있는 긴 text들 가져오는 것
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)

    return cleaned_docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # 특수 문자를 추가해보자.
    for i in string.punctuation: # punctuation : 특수문자
        SW.add(i) # 굳이 써준 이유는 이렇게도 쓸 수 있다는 것 보기위해
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path, encoding='UTF-8') as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    # 명사 추출 / 형태소 분석 두 가지를 선택할 수 있게 만들어주는 함수를 만들어보자.
    # 좀 지져분한 텍스트 경우 명사 추출도 많이 쓰임(키워드를 추출한다는 느낌)
    token_corpus = []
    
    # tqdm을 사용하여 진행 과정을 볼 수 있게 만들어보자.
    # tokenizer변수는 어떤 기준으로 자를 것인지 나타냄
    
    if tokenizer == 'noun': # 명사 추출
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n]) # 명사추출
            token_text = [word for word in token_text if word not in SW and len(word) > 1] # 사람마다 규칙은 다른데 1글자는 안쓰겠다는 것
            
            token_corpus.append(token_text)
            
    elif tokenizer == 'morph': # 형태소 분석
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            
            token_corpus.append(token_text)
            
    elif tokenizer == 'word ': # 만약 워드 단위면
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split() # 워드 단위면 그냥 split해주겠다는 거니까
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            
            token_corpus.append(token_text)
            
    return token_corpus

# 함수를 불러오는 (메인) 코드.
input_file_name = "data/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("data/stopwords-ko.txt")
cleaned_text = text_cleaning(documents) # 얘는 명사추출만 할거면 안 써도 돼(어차피 명사만 뽑힐거니까)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "morph" or "word"

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [30]:
# 결과 확인.
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '정부', '디지털', '뉴딜', '뉴딜', '국판', '뉴딜', '투입', '사진', '홍남기', '경제', '부총리', '기획', '재정부', '장관', '이동률', '기자', '정부', '국판', '뉴딜', '디지털', '그린', '투입', '이성락', '기자', '포스트', '코로나', '시대', '대비', '장기', '한국', '경제', '도약', '준비', '프로젝트', '국판', '뉴딜', '투입', '정부', '발표', '국판', '뉴딜', '디지털', '뉴딜', '뉴딜', '고용', '안전망', '강화', '토대', '추진', '이날', '정부', '기획', '재정부', '관계', '부처', '합동', '발표', '하반기', '경제', '정책', '방향', '경정', '국판', '뉴딜', '추진', '방침', '발표', '정부', '국판', '뉴딜', '추진', '문재', '정부', '임기', '조억', '투자', '수준', '일자리', '창출', '계획', '국판', '뉴딜', '완성', '문재인', '정부', '이후', '추가', '투입', '국가', '산업', '재편', '디지털', '강국', '입지', '방침', '구체', '디지털', '뉴딜', '중점', '과제', '생태', '강화', '디지털', '포용', '안전망', '구축', '산업', '육성', '사회', '간접', '자본', '디지털', '정부', '조억', '투입', '생태', '강화', '데이터', '네트워크', '인공지능', '사업', '일자리', '창출', '계획', '정부', '금융', '환경', '문화', '교통', '헬스', '케어', '국민', '생활', '밀접', '분야', '데이터', '플랫', '구축', '공공', '데이터', '개방', '학습', '데이터', '추가', '구축', '조기', '구축', '융합', '산업', '활성', '국가', '클라우드', '전

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [31]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text) # 리스트 형태의 input, tokenized_text는 list of list of word지

# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]
# doc2bow : bow로 바꿔줌, sklearn의 countvectorizer랑 똑같음

- LDA의 input으로 들어가는 게 크게 dictionary랑 corpus! 꼭 들어가야 해
- LDA도 input이 document-term-matrix(문서-단어 행렬)라서 bow(bag-of-word)표현으로 바꿔줘야해
- dictionary:어떤 단어가 몇번째를 가지는지 (해쉬테이블), key값이 숫자, value가 단어 => 이 구조를 만들어줘야해 컴퓨터는 이 단어가 어디에 있는지 모르니까 효율적으로 빠르게해주기위해

In [33]:
# Dictionary 확인
print(dictionary) # 가나다 순 정렬
# dictionary라는 것은 전체 단어를 다 가지고있는데 단어마다 각자 인덱스가 붙어있는 구조

Dictionary(1246 unique tokens: ['가능', '가운데', '간선망', '간접', '감면']...)


In [34]:
# corpus 확인
corpus[0][:5]


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [36]:
corpus[0] # 하나하나 원소가 1개, 1개, 3개, 6개 등 이런식으로 있다
# 앞에 붙은 숫자는 dictionary 인덱스

# bow표현으로 dictionary를 만들었는데, 어떤 차원의 어떤 단어인지 알아야하는데 그걸 dictionary란 구조로 가지고있는 것
# corpus는 input으로 넣은 text를 각 dictionary에 맞는 구조로 바꿔준 것
# bow에 안에 들은 원소값이 있거나 없거나가 아니고, word frequency가 들어있음

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 3),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 6),
 (19, 1),
 (20, 1),
 (21, 3),
 (22, 2),
 (23, 3),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 3),
 (29, 1),
 (30, 3),
 (31, 3),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 2),
 (36, 1),
 (37, 3),
 (38, 1),
 (39, 14),
 (40, 4),
 (41, 1),
 (42, 2),
 (43, 9),
 (44, 2),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 4),
 (51, 2),
 (52, 1),
 (53, 4),
 (54, 2),
 (55, 1),
 (56, 2),
 (57, 5),
 (58, 2),
 (59, 4),
 (60, 7),
 (61, 2),
 (62, 16),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 2),
 (67, 3),
 (68, 5),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 3),
 (73, 1),
 (74, 3),
 (75, 1),
 (76, 10),
 (77, 1),
 (78, 1),
 (79, 2),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 2),
 (89, 1),
 (90, 4),
 (91, 1),
 (92, 2),
 (93, 1),
 (94, 3),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 

In [38]:
# TFIDF 문서-단어 행렬 생성

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.012769970398895393),
 (1, 0.029650953040075187),
 (2, 0.029650953040075187),
 (3, 0.029650953040075187),
 (4, 0.04242092343897059)]

In [39]:
# LDA model 만들기

# models.ldamodel.LdaModel(input, 토픽 몇개 할건지, 연결고리인 dictionary(워드가 어디에있냐))
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [40]:
# LDA 결과 확인

# model.show_topic(topic_no, num_words) 
model.show_topic(0, 10) # 0번 토픽에 10개를 보고싶어
# ('단어', '확률')

[('데이터', 0.009317247),
 ('플랫', 0.008084395),
 ('기술', 0.0073465467),
 ('장비', 0.0072188983),
 ('산업', 0.0071635167),
 ('내용', 0.0069993124),
 ('서비스', 0.005873619),
 ('개발', 0.0058664237),
 ('플레이어', 0.005758702),
 ('디지털', 0.0056109247)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [41]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3 # k개

NUM_TOPIC_WORDS = 30 # 몇개로 짜를거냐

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
    
    return corpus, dictionary

def print_topic_words(model):
    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    # 결과를 전부 다 보고싶어
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)

# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
# alpha="auto", eta="auto" 파라미터 쓰면 좋음

# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	대학	0.01244397647678852
	디지털	0.011410382576286793
	교육	0.010093418881297112
	지원	0.0100662000477314
	온라인	0.008152097463607788
	운영	0.007533877622336149
	구축	0.007348949555307627
	학습	0.007184790447354317
	기술	0.006940205581486225
	산업	0.006387542001903057
	개발	0.006160398479551077
	플랫	0.0060598221607506275
	프로그램	0.005820531863719225
	부산	0.005545479245483875
	데이터	0.005428755655884743
	콘텐츠	0.005358814261853695
	플레이어	0.005249925889074802
	사업	0.005239660385996103
	제공	0.005209360271692276
	내용	0.005153107922524214
	장비	0.005040039774030447
	서비스	0.00480011198669672
	한국	0.004758124705404043
	러닝	0.0047540548257529736
	스튜디오	0.004635865334421396
	학생	0.004485586192458868
	시스템	0.004318140912801027
	이스라엘	0.004239083267748356
	뉴딜	0.004168623127043247
	본문	0.004068742040544748


Topic ID: 1
	기술	0.013244662433862686
	이스라엘	0.011080932803452015
	한국	0.010493958368897438
	청약	0.007960664108395576
	기업	0.007074072491377592
	사업	0.006959413178265095
	시장	0

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [42]:
# !pip install pyLDAvis

# LDAvis는 R언어를 위한 패키진데 pyLDAvis는 python에서 LDA결과 시각화 해주는 패키지

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97716 sha256=1250220fd8f02048468d69ce78ea2172087fdec6519417d631cfc4a5097d8028
  Stored in directory: c:\users\영현\appdata\local\pip\cache\wheels\57\de\11\0a038be70c2c212ce45fa0f4f9da165bb5dd87de1288394dc3
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32045 sha256=17460c0a8a93d20357ee6f0c6fbcc8e3a03dafdbaa0a85e431cf8aae2ab24db5
  Stored in directory: c:\users\영현\appdata\local\pip\cache\wheels\5e\e0\ba\0dd7bb3f79264f8f60690da62918081bb8c3fb7442c38bfddd
Successfully built pyLDAvis funcy


In [43]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data # print치면 안돼

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.023048  0.011852       1        1  40.702103
2     -0.008328 -0.016246       2        1  30.897102
0      0.031376  0.004394       3        1  28.400803, topic_info=     Term       Freq      Total Category  logprob  loglift
582    대학  16.000000  16.000000  Default  30.0000  30.0000
35     교육  14.000000  14.000000  Default  29.0000  29.0000
358   온라인  11.000000  11.000000  Default  28.0000  28.0000
257    학습  10.000000  10.000000  Default  27.0000  27.0000
525    운영  11.000000  11.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
31     관리   3.653595  10.211226   Topic3  -5.5902   0.2310
103    본문   3.980888  13.637668   Topic3  -5.5044   0.0274
258    한국   4.655385  21.988503   Topic3  -5.3479  -0.2937
987  이스라엘   4.147551  21.038691   Topic3  -5.4634  -0.3651
62     뉴딜   4.078612  20.695620   Topic3  -5.4802  -0.3654

[208 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1116      1  0.528014   가구
1116      2  0.316808   가구
1116      3  0.105603   가구
5         1  0.181894  감염병
5         2  0.545681  감염병
...     ...       ...  ...
264       3  0.210157   혁신
266       1  0.314404  홍남기
266       2  0.628809  홍남기
266       3  0.314404  홍남기
719       3  0.498251   화상

[375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

- 왜 top에 있는 단어들이 비슷비슷할까?
    - 기사가 최신순이고 몇 개 없어서 그럼. 텍스트가 비슷비슷한거지
    - 기사 개수가 많아지고 연도도 다양해지면 더 다양한 토픽들이 있을 수 있겠지

In [ ]:
# pc1, pc2 해석 들으려면 강의 다시 듣자